In [0]:
%sql
USE CATALOG spotify_etl;
USE SCHEMA gold

In [0]:
from pyspark.sql.functions import col, avg, count,split, expr, round,collect_set

In [0]:
df_gold = spark.table("spotify_etl.silver.spotify_tracks_cleaned")


In [0]:
# Tabelas de Dimensão
# =========================

# Dimensão Artista
df_dim_artist = df_gold.select("artists_ids", "artist_names") \
    .dropDuplicates() \
    .withColumnRenamed("artists_ids", "id_artist")

df_dim_artist.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.dim_artist")

# Dimensão Tempo
df_dim_tempo = df_gold.select(
    col("id").alias("id_musica"),
    "duration_min",
    "tempo",
    "loudness"
).dropDuplicates()

df_dim_tempo.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.dim_tempo")

# Dimensão Acústica
df_dim_acustica = df_gold \
    .select(
        col("id").alias("id_musica"),
        "name",
        "artist_names",
        "acousticness",
        "instrumentalness",
        "is_acoustic",
        "is_instrumental",
        "is_energy"
    ) \
    .dropDuplicates()


df_dim_acustica.write.format("delta").mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("spotify_etl.gold.dim_acustica")

# Dimensão Sentimento
df_dim_sentimento = df_gold.select(
    col("id").alias("id_musica"),
    "valence",
    "sentimento"
).dropDuplicates()

df_dim_sentimento.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.dim_sentimento")

# Dimensão Musicalidade
df_dim_musicalidade = df_gold.select(
    col("id").alias("id_musica"),
    "key_note",
    "modo",
    col("time_signature").alias("compasso")
).dropDuplicates()

df_dim_musicalidade.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.dim_musicalidade")

# Dimensão Gênero
df_dim_genero = df_gold.select(
    col("id").alias("id_musica"),
    col("genres").alias("generos")
).dropDuplicates()

df_dim_genero.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.dim_genero")

# Tabela Fato
# =========================

df_fato_music = df_gold.select(
    col("id").alias("id_musica"),
    col("artists_ids").alias("id_artist"),
    "energy",
    "valence",
    "acousticness",
    "duration_min",
    "loudness",
    "danceability"
) \
.join(df_dim_sentimento.select("id_musica"), on="id_musica", how="left") \
.join(df_dim_acustica.select("id_musica"), on="id_musica", how="left") \
.join(df_dim_genero.select("id_musica"), on="id_musica", how="left") \
.join(df_dim_tempo.select("id_musica"), on="id_musica", how="left") \
.join(df_dim_musicalidade.select("id_musica"), on="id_musica", how="left") \
.dropDuplicates(["id_musica"])


df_fato_music.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("spotify_etl.gold.fato_music")



In [0]:
df = df_gold.withColumn("genero_principal", split(col("genres"), ",")[0])

df_dm_perfil_artista = df.groupBy("artist_names") \
    .agg(
        count("*").alias("total_musicas"),
        round(avg("valence"), 2).alias("valencia_media"),
        expr("first(genero_principal)").alias("genero_principal"),
        expr("first(sentimento)").alias("sentimento_predominante"))
    
df_dm_perfil_artista.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.df_dm_perfil_artista")

In [0]:
df_genero_dancante = df_fato_music.join(df_dim_genero, "id_musica") \
    .groupBy("generos") \
        .agg(avg("danceability").alias("media_danca")) \
            .orderBy("media_danca", ascending=False)

df_genero_dancante.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("spotify_etl.gold.genero_dancante")

In [0]:
from pyspark.sql.functions import avg, collect_set, explode

df_genero_dancante_com_artistas = df_fato_music \
    .join(df_dim_genero, "id_musica") \
    .join(df_dim_artist, "id_artist") \
    .groupBy("generos") \
    .agg(
        avg("danceability").alias("media_danca"),
        collect_set("artist_names").alias("artistas")
    ) \
    .orderBy("media_danca", ascending=False)

df_genero_dancante_exploded = df_genero_dancante_com_artistas \
    .withColumn("artista", explode("artistas")) \
    .drop("artistas")  # remove a coluna de array original, se quiser

df_genero_dancante_exploded.write.format("delta").mode("overwrite") \
    .saveAsTable("spotify_etl.gold.genero_dancante_com_artistas")


In [0]:
df_genero_dancante_exploded.display()

generos,media_danca,artista
"latin pop,pop rap,pop,dance pop,colombian pop",0.964,"Black Eyed Peas, Shakira"
"brega funk,funk carioca,funk mtg",0.96,"MC Keron, MC Digu"
"funk ostentacao,funk carioca,funk mandelao",0.956,"MC Rafa 22, DJ Wallace NK"
"funk viral,funk 150 bpm,funk mtg,rave funk,funk rj",0.955,"Dj JL O Único, Dj LK da Escócia, DJ Breno, DJ Pedrin, Dj Créu"
"pop nacional,funk rj,funk carioca,brazilian hip hop",0.955,"Papatinho, Luísa Sonza, DJ Biel do Furduncinho"
"funk carioca,funk mtg,funk pop",0.953,"MC Levin, Caverinha"
"funk carioca,pop rap brasileiro,pop nacional,trap brasileiro,funk rj",0.953,"L7NNON, DJ Biel do Furduncinho, Bianca"
"deboxe,funk viral",0.953,"MC Kaique da VP, DJ SKYPE"
"funk carioca,funk viral,funk bh,deboxe,funk paulista,funk mtg,funk ostentacao,funk rj",0.95,"DJ JR Oficial, GORDÃO DO PC, DJ Lg do Sf, MC Theuzyn, MC Teteu, Mc Talibã, MC Lil, Mc J Mito"
"funk carioca,funk paulista,funk mtg,funk mandelao,funk rj",0.949,"mc jhenny, DJ KOSTA 22, Mc Gw"


In [0]:
# playlist de musica acustica
df_playlist_acoustic = df_fato_music.join(df_dim_acustica, "id_musica") \
    .filter("is_acoustic = true") \
    .select("id_musica", "name", "artist_names", "duration_min", "valence", "energy")

df_playlist_acoustic.write.mode("overwrite")\
    .option("overwriteSchema", "true")\
    .saveAsTable("spotify_etl.gold.playlist_acustica_recomendada")

In [0]:
# quantidades de musicas acusticas por tom
df_acoustic_por_tom = df_gold.filter("is_acoustic = true") \
    .groupBy("key_note") \
        .count() \
            .orderBy("count", ascending=False)

In [0]:
df_artistas_mais_presentes = df_gold.groupBy("artist_names") \
    .agg(
        count("*").alias("qtd_musicas"),
        avg("energy").alias("media_energia"),
        avg("valence").alias("media_valence"),
        avg("danceability").alias("media_danceabilidade")
    ) \
    .orderBy("qtd_musicas", ascending=False)

df_artistas_mais_presentes.write.mode("overwrite").saveAsTable("spotify_etl.gold.artistas_mais_presentes")

In [0]:
from pyspark.sql.functions import col, split, count

df_artistas_mais_presentes_por_genero = df_gold \
    .withColumn("genero", split(col("genres"), ",").getItem(0)) \
    .groupBy("artist_names", "genero") \
    .agg(
        count("*").alias("qtd_musicas")
    ) \
    .orderBy("qtd_musicas", ascending=False)

df_artistas_mais_presentes_por_genero.write.mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("spotify_etl.gold.df_artistas_mais_presentes_por_genero")

df_artistas_mais_presentes_por_genero.display()



artist_names,genero,qtd_musicas
Henrique & Juliano,agronejo,124
Taylor Swift,pop,80
Zé Neto & Cristiano,agronejo,77
Marília Mendonça,sertanejo universitario,72
Gusttavo Lima,agronejo,71
Nadson O Ferinha,forro,60
Jorge & Mateus,agronejo,54
Matheus & Kauan,agronejo,53
Luísa Sonza,pop nacional,40
Wesley Safadão,sertanejo universitario,37
